In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors
import collections
import seaborn as sns

In [2]:
# custom helper functions
import carbohydrate_helpers as carbo
import plot_helpers as plot

In [3]:
# data tables
EC_df = pd.read_table('data-tables/CAZyme_EC_activities.txt') # EC activities found in CAZymes with manually annotated substrates & activity
carbo_df = pd.read_table('data-tables/carbohydrates.txt') # carbohydrates and glycosides with manually annotated structures
cazyfam_df = pd.read_table('data-tables/CAZy_fams.txt', index_col=0) # cazyme family vs EC activities and CAZydb description
cazyme_df = pd.read_csv('/home/ecutts/sharkBayData/cazymes/results/dbCAN_signalp_cazyfam_vs_mag.csv', index_col=0) # CAZyme counts by family for each MAG (degradation CAZymes only)
cazyme_ex_df = pd.read_table('/home/ecutts/sharkBayData/cazymes/results/dbCAN_signalp_cazyfam_vs_mag_ex.csv', index_col=0) # Extracellular CAZyme counts by family for each MAG (degradation CAZymes only)
metadata = pd.read_csv('data-tables/mags-meta.csv', index_col=0) # MAG metadata

# Substrate auto-mapping
Automatically assign substrates and substrate metadata to CAZyme families based on the `carbohydrates.txt` file, and map these data to the MAGs. 

The accessory functions used in this section are found in `carbohydrate_helpers.py` under the heading `AUTOMAPPING`. 

In [4]:
# map substrates and activity to CAZymes
cazyfam_df = carbo.map_substrates_and_activity(cazyfam_df, carbo_df, EC_df)

# add information from a column in carbo_df to a dataframe based on its substrates column.
cazyfam_df = carbo.add_substrate_metadata_cols(cazyfam_df, carbo_df)

# add count info
def get_count_from_fam(fam, cazyme_df):
    if fam in cazyme_df.columns:
        return(cazyme_df[fam].sum())
    else:
        return(0)

cazyfam_df['count'] = cazyfam_df['family'].apply(get_count_from_fam, args=(cazyme_df,))
cazyfam_df['ex_count'] = cazyfam_df['family'].apply(get_count_from_fam, args=(cazyme_ex_df,))


cazyfam_df.to_csv('data-tables/CAZy_fams_autoannos.txt')
cazyfam_df[cazyfam_df['count'] > 0].to_csv('data-tables/CAZy_fams_autoannos_presentInMAGs.txt')

# Manually refine auto-mapped spreadsheet and re-import

After creating the automatically generated substrate annotations, go through the spreadsheet and look for cazyme families that were missed by automatic annotation. Some families  don't have EC numbers associated with them, but still have a well-described activity that can be manually added.

After manual refinement, re-import the spreadsheet and create a dataframe containing all information about all CAZymes in every MAG. Each row represents the genes in a given family in a given MAG. The `accession` column gives the MAG, the `family` column gives the CAZyme family, and the columns `count` and `ex_count` give the number of genes in the family in that MAG.

Relevant functions found in `carbohydrate_helpers.py`

In [5]:
cazyfam_df = pd.read_table('data-tables/CAZy_fams_refined_presentInMAGs.txt')
cazyfam_df = carbo.add_substrate_metadata_cols(cazyfam_df, carbo_df)
combined_cazyfam_df = carbo.make_combined_cazyfam_df(cazyfam_df, cazyme_df, cazyme_ex_df)
combined_cazyfam_df.to_csv('data-tables/CAZy_fams_combined_MAGs.tsv', sep='\t')

# Create tables of substrates and bonds targeted by GH and PL that appear in the MAGs

GH and PL cleave glycosidic linkages and can therefore be described in common terms, making it easier to lump them together for analysis than it would be to include CE and other families that have different kinds of activity. 

This code creates two spreadsheets: `shark-bay-substrates.csv` and `shark-bay-bonds.csv`. These are manually annotated in the next step and given categories for analysis. The category assignment is manual, since I thought it was best to look at the diversity of bonds present before deciding which to bin together. 

One could do this for all substrates and bonds targeted by all CAZymes, but to save time I only addressed those substrates and bonds targeted by CAZymes in my dataset. I think that taking a look at the targets of your dataset specifically is also a good exercise in data exploration.

In the future, it would be best to extend the spreadsheet I made here to include all bonds and substrates targeted by CAZymes in CAZydb, so that this process could be skipped and the same spreadsheet could be used in all analyses.

In [6]:
ghpl_df = cazyfam_df[(cazyfam_df['family'].str.contains('GH') | cazyfam_df['family'].str.contains('PL'))]
substrates = collections.Counter(carbo.get_list_from_col(ghpl_df, 'substrates', count_col='count'))
ex_substrates = collections.Counter(carbo.get_list_from_col(ghpl_df, 'substrates', count_col='ex_count'))

# add in missing (0 count) substrates from ex_counts
for key in substrates.keys():
    if key not in ex_substrates.keys():
        ex_substrates[key] = 0
        
        

# add make substrates dataframe and export

substrates = dict(sorted(substrates.items(), key=lambda item: item[1], reverse=True))
ex_substrates = dict(sorted(ex_substrates.items(), key=lambda item: item[1], reverse=True))
sb_substrates = pd.DataFrame({'count': substrates.values(), 'ex_count': ex_substrates.values()}, index=substrates.keys())
sb_substrates['delta'] = sb_substrates['count'] - sb_substrates['ex_count']

mags = []
ex_mags = []
for substrate in substrates.keys(): 
    families = carbo.get_cazyfams_from_substrate(substrate, ghpl_df, 'substrates')
    magids = []
    ex_magids = []
    for i in families:
        if i in cazyme_df.columns:
            magids += list(cazyme_df[cazyme_df[i] != 0].index)
        if i in cazyme_ex_df.columns:
            ex_magids += list(cazyme_ex_df[cazyme_ex_df[i] != 0].index)
    mags.append(len(set(magids)))
    ex_mags.append(len(set(ex_magids)))

sb_substrates['mags'] = mags
sb_substrates['ex_mags'] = ex_mags
sb_substrates['superfamily'] = pd.Series(sb_substrates.index).apply(carbo.get_substrate_info, args=('superfamily',carbo_df,)).tolist()
sb_substrates['family'] = pd.Series(sb_substrates.index).apply(carbo.get_substrate_info, args=('family',carbo_df,)).tolist()
sb_substrates['subfamily'] = pd.Series(sb_substrates.index).apply(carbo.get_substrate_info, args=('subfamily',carbo_df,)).tolist()
sb_substrates.to_csv('data-tables/shark-bay-substrates.csv')



# make bonds dataframe and export

sb_activities = ghpl_df['bond activity'].str.split(' ')
sb_activities_df = pd.DataFrame({1:sb_activities.str[0], 2:sb_activities.str[1],
                                 3:sb_activities.str[2], 4:sb_activities.str[3]})

sb_activities_df = sb_activities_df[sb_activities_df[3].notna()]
sb_activities_df[4] = sb_activities_df[4].str.split(';').str[0]
sb_activities_df = sb_activities_df[sb_activities_df[3] != 'to']
sb_bonds = set(sb_activities_df[2] + " " + sb_activities_df[3] + " " + sb_activities_df[4])

ex_combined_cazyfam_df = combined_cazyfam_df[combined_cazyfam_df['ex_count'] != 0]

combined_mag_ghpl_df = combined_cazyfam_df[combined_cazyfam_df['family'].str.contains('GH') | combined_cazyfam_df['family'].str.contains('PL')]
ex_combined_mag_ghpl_df = combined_mag_ghpl_df[combined_mag_ghpl_df['ex_count'] != 0]
mags = []
ex_mags = []
count = [] # COUNT HAS A PROBLEM 
ex_count = []
sb_bonds = [x for x in sb_bonds if str(x) != 'nan']
for i in sb_bonds:
    rows_with_bond = combined_mag_ghpl_df[combined_mag_ghpl_df['bond activity'].str.contains(i, regex=False) == True]
    ex_rows_with_bond = ex_combined_mag_ghpl_df[ex_combined_mag_ghpl_df['bond activity'].str.contains(i, regex=False) == True]
    mags.append(len(set(rows_with_bond['accession'])))
    ex_mags.append(len(set(ex_rows_with_bond['accession'])))
    count.append(rows_with_bond['count'].sum())
    ex_count.append(ex_rows_with_bond['ex_count'].sum())

sb_bonds_df = pd.DataFrame({'count': count, 'ex_count': ex_count, 'mags': mags, 'ex_mags': ex_mags}, index=sb_bonds)
sb_bonds_df.to_csv('data-tables/shark-bay-bond-targets.csv', sep='\t')

# Manually annotate the bonds and substrates tables

The next step is to manually annotate the bonds and substrates tables. Give bonds and substrates categories (`cat1`, `cat2`, `cat3`, with each higher number category being a more specific sub-category) that you want to use for analysis. I do this manually because looking at which bonds/substrates are present is important for deciding how to bin them together. 

Examples of manually-annotated bonds and substrates tables are available in the `data-tables` folder as `shark-bay-bonds-manualannos.tsv` and `shark-bay-substrates-manualannos.tsv`.

Of course, making a bonds and substrates table that works for all CAZymes would be better. But that's beyond the scope of this project.

# Import manually annotated bonds and substrate tables and add that information to `combined_cazyfam_df` as dictionaries

In [7]:
sb_bonds_df = pd.read_table('data-tables/shark-bay-bonds-manualannos.tsv')
sb_subs_df = pd.read_table('data-tables/shark-bay-substrates-manualannos.tsv')
    
combined_cazyfam_df['substrate cat1'] = combined_cazyfam_df['substrates'].apply(carbo.get_categories_from_activity_string, args=('cat1',sb_subs_df, ), return_strlist=True)
combined_cazyfam_df['substrate cat2'] = combined_cazyfam_df['substrates'].apply(carbo.get_categories_from_activity_string, args=('cat2',sb_subs_df, ), return_strlist=True)
combined_cazyfam_df['activity cat1'] = combined_cazyfam_df['bond activity'].apply(carbo.get_categories_from_activity_string, args=('cat1',sb_bonds_df, ), return_strlist=True)
combined_cazyfam_df['activity cat2'] = combined_cazyfam_df['bond activity'].apply(carbo.get_categories_from_activity_string, args=('cat2',sb_bonds_df, ), return_strlist=True)

combined_cazyfam_df.to_csv('data-tables/CAZy_fams_combined_MAGs.tsv', sep='\t')

### Anaylysis continues in `explore-cazymes.ipynb`